In [13]:
!pip3 uninstall numpy


zsh:1: command not found: Y
ERROR: Pipe to stdout was broken


In [1]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens1MReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/data') #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

ModuleNotFoundError: No module named 'Data_manager'

In [2]:
import optuna
import numpy as np
import pandas as pd
import os

def create_objective_d(recommender_instance, URM_train, URM_validation, hyperparam_space, evaluator_validation, cutoff, directory_path):
    def objective(trial):
        # Gather and set hyperparameters
        hyperparams = {name: trial.suggest_categorical(name, space) for name, space in hyperparam_space.items()}

        # Initialize and train the recommender
        
        recommender_instance.fit(**hyperparams)

        # Evaluate recommender
        user_id_array = np.arange(URM_validation.shape[0])
        
        result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
        result_df['hyperparams'] = str(hyperparams)

        # Assuming df_row is the DataFrame containing the row to append

        filename = directory_path + recommender_instance.RECOMMENDER_NAME + ".csv"

        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)
    
        return result_df.loc[cutoff]['NDCG']

    return objective


In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

PyTorch version: 2.3.0.dev20240128
Is MPS (Metal Performance Shader) built? True
tensor([1.], device='mps:0')


In [2]:
from Diffusion.DiffusionRecommender import SimpleAttentionDiffusionRecommender
diffusion_model = SimpleAttentionDiffusionRecommender(URM_train = URM_train, use_gpu = True)
diffusion_model.fit(epochs=50)

ModuleNotFoundError: No module named 'Diffusion'